# xdggs: using discrete global grid systems with `xarray`

`xdggs` uses the information contained in the cell ids to perform operations on the cell ids.

It currently includes built-in support for:
- healpix (using `cds-healpix-python`)
- h3 (using `h3ronpy`)

Any operations on the data will then use standard `xarray` methods.

Github: https://github.com/xarray-contrib/xdggs

Documentation: https://xdggs.readthedocs.io

Install from PyPI:
```sh
pip install xdggs
```

(Soon on `conda-forge` as well)

## opening data

`xdggs` only works for data already on a DGGS. If that is not already the case, the dataset will need to be regridded first.

```{note}
`xdggs` explicitly excludes regridding from / to a DGGS, as that is better solved by a separate, more specialized package.

In the following, we'll use the `air_temperature` dataset from `xarray`'s tutorial data that has already been interpolated to a DGGS. We'll also choose `h3` for demonstration purposes, but you can choose any of the supported DGGS and the code will work in the same way.
```

In [ ]:
import xarray as xr
import xdggs

xr.set_options(display_expand_data=False, display_expand_attrs=False);

In [ ]:
ds = xdggs.tutorial.open_dataset("air_temperature", "h3").load()
ds

In [ ]:
# open the data
ds = xdggs.tutorial.open_dataset("air_temperature", "h3").load()
ds

To then interact with the cell ids, we need to create an index on the cell ids:

```{note}
This makes use of the attributes on the `cell_ids` variable. The required attributes are:
- `grid_name`: the name of the grid
- `level`: the grid hierarchical level. Corresponds to the resolution of the grid.

Grids can define more parameters, like the indexing scheme for `healpix`.

Note that currently the variable name is hard-coded to `cell_ids`, but this will be customizable (and possibly auto-detected) in the future.
```

In [ ]:
ds = ds.pipe(xdggs.decode)
ds

In [ ]:
# create the index
ds = ds.pipe(xdggs.decode)
ds

## plotting

To get a good idea of what the data is about, we need to plot. For interactive visualization, `xdggs` interfaces with [lonboard](https://github.com/developmentseed/lonboard):

```{note}
This is currently restricted to 1D `DataArray`s, but in the future will include a slider for additional dimensions.
```

In [ ]:
ds["air"].isel(time=15).dggs.explore(alpha=0.8, center=273.15, cmap="coolwarm")

In [ ]:
# visualize the data
ds["air"].isel(time=15).dggs.explore(alpha=0.8, center=273.15, cmap="coolwarm")

## selecting data

Since the cell ids contain information about the cells and their extent, we can subset the data using the index.

```{note}
At the moment, we can only select individual points (`xarray` calls this [vectorized indexing](https://docs.xarray.dev/en/stable/user-guide/indexing.html#vectorized-indexing)), but there are plans to extend this to bounding boxes or arbitrary geometries.
```

In [ ]:
data = ds.drop_vars(["lat", "lon"])

lat = xr.DataArray([24, 30, 40, 50, 60], dims="points")
lon = xr.DataArray([293, 240, -100, -80, -60], dims="points")

In [ ]:
subset = data.dggs.sel_latlon(latitude=lat, longitude=lon)
subset

In [ ]:
subset["air"].isel(time=15).dggs.explore(alpha=0.8, center=273.15, cmap="coolwarm")

In [ ]:
# select using points
subset = ds.dggs.sel_latlon(latitude=lat, longitude=lon)
subset["air"].isel(time=15).dggs.explore(alpha=0.8, center=273.15, cmap="coolwarm")

## deriving information

To interact with other libraries that don't know how to work with cell ids, we need to be able to compute the cell center coordinates and cell boundaries.

### cell centers

In [ ]:
expected = (
    ds[["lat", "lon"]]
    .rename({"lat": "latitude", "lon": "longitude"})
    .drop_vars("cell_ids")
)

In [ ]:
actual = data.dggs.cell_centers()
xr.testing.assert_equal(actual, expected)

In [ ]:
# compute cell centers
cell_centers = ds.dggs.cell_centers()

xr.testing.assert_allclose(cell_centers, expected)

### cell boundaries

In [ ]:
data.dggs.cell_boundaries()

In [ ]:
# compute cell boundaries
cell_boundaries = ds.dggs.cell_boundaries()
cell_boundaries

## future work

There are still a lot of exciting new features planned (as time permits):
- hierarchy operations:
    - compute parents / children / siblings
    - neighbours search
- indexing using:
    - geometries (polygon, circle, ...)
    - bounding box
- grid alignment ("overlap between two grids")
- plotting:
    - allow plotting n-dimensional data using sliders
    - more customization (e.g. colorbar)
- plugin system for third-party DGGS implementations:
    - publicly expose and document the plugin system
- interface with other libraries (`xvec`, `uxarray`, ...)